In [1]:
import json
import timetable_scheduler
from timetable_scheduler.simulated_annealing import CoolingSchedule, quadratic_cooling_schedule, \
    exponential_cooling_schedule, linear_cooling_schedule, logarithmic_cooling_schedule, bolzmann_cooling_schedule, \
    cauchy_cooling_schedule

COOLING_SCHEDULE_MAP = {
    'exponential': exponential_cooling_schedule,
    'linear': linear_cooling_schedule,
    'logarithmic': logarithmic_cooling_schedule,
    'quadratic': quadratic_cooling_schedule,
    'bolzmann': bolzmann_cooling_schedule,
    'cauchy': cauchy_cooling_schedule
}

PARAMETERS_SETUP = {
    exponential_cooling_schedule: ([25, 25, 30],[2.03, 0.166, 0.00133], [30, 30, 30], [0.995, 0.995, 0.995]),
    linear_cooling_schedule: ([15, 15, 15],[0.15, 0.15, 0.15], [100, 100, 100], [0.02, 0.01, 0.005]),
    logarithmic_cooling_schedule: ([30, 30, 30],[0.483, 0.434, 0.395], [10, 10, 10], [10, 10, 10]),
    quadratic_cooling_schedule: ([1000, 1000, 1000],[0.004, 0.004, 0.004], [1, 1, 1], [1, 0.22, 0.06]),
    bolzmann_cooling_schedule: ([50, 50, 50],[6.93, 6.32, 5.81], [20, 20, 20], [1, 1, 1]),
    cauchy_cooling_schedule: ([50, 50, 50],[0.1, 0.05, 0.025], [30, 40, 50], [1, 1, 1]),
}

iterations = 0
@timetable_scheduler.timing
def test_instance_creator(term_id: int, availability_p: float, cooling_schedule: CoolingSchedule, ):
    results = {}
    global iterations
    n_tests = 25
    timetable_scheduler.create_dataset(term_id=term_id, lecturer_p=availability_p, room_p=availability_p)

    tmaxes, tmins, kmaxes, alphas = PARAMETERS_SETUP[cooling_schedule]

    for Tmax, Tmin, kmax, alpha  in zip(tmaxes, tmins, kmaxes, alphas):
        results[(Tmax, Tmin, kmax, alpha)] = []
        setup = timetable_scheduler.StatisticalTestsAlgorithmSetup(Tmax=Tmax, Tmin=Tmin, kmax=kmax, alpha=alpha, cooling_schedule=cooling_schedule)
        for i in range(n_tests):
            result = setup.SA()
            results[(Tmax, Tmin, kmax, alpha)].append(result)
            iterations += 1
        print('-'*10, iterations, '-'*10)
    return results



In [2]:
time_performance = []
for name, cooling_schedule in COOLING_SCHEDULE_MAP.items():
    results, time = test_instance_creator(term_id=5, availability_p=1, cooling_schedule=cooling_schedule)
    time_performance.append(f'{name}-{time}s')
    json_data = {str(parameters): [r.to_json() for r in result] for parameters, result in results.items()}
    with open(f'statistics/{name}_cooling_schedule_test.json', 'w') as f:
        json.dump(json_data, f, indent=4)

zresetowałem początkowe rozwiązanie 0-razy
1807.6000000000001
25
21.5096047978674
18.506523942417612
15.922720647346953
13.699657137251053
11.786968435542144
10.141321312537958
8.725432542435255
7.507224227129348
6.459097050181113
5.557304996018673
4.781417368223029
4.113855918565465
3.539496600148436
3.0453269221035266
2.6201511429901108
2.2543366238559135
Best cost = 1444.1999999999998
zresetowałem początkowe rozwiązanie 0-razy
1763.1999999999998
25
21.5096047978674
18.506523942417612
15.922720647346953
13.699657137251053
11.786968435542144
10.141321312537958
8.725432542435255
7.507224227129348
6.459097050181113
5.557304996018673
4.781417368223029
4.113855918565465
3.539496600148436
3.0453269221035266
2.6201511429901108
2.2543366238559135
Best cost = 1365.6
zresetowałem początkowe rozwiązanie 0-razy
1965.3999999999999
25
21.5096047978674
18.506523942417612
15.922720647346953
13.699657137251053
11.786968435542144
10.141321312537958
8.725432542435255
7.507224227129348
6.459097050181113

In [3]:
for stat in time_performance:
    print(stat)

exponential-1272.3146135807037s
linear-1893.7485346794128s
logarithmic-1185.5440266132355s
quadratic-2028.3099999427795s
bolzmann-1320.8370077610016s
cauchy-1363.0260882377625s


In [12]:
total_time = sum([float(stat.split('-')[1][:-1]) for stat in time_performance]) / 3600
print(f'total tests runtime = {total_time:.2f}h')

total tests runtime = 2.52h
